In [ ]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year


In [ ]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass")
mature_biomass_area_2020 = ee.Image(f"{data_folder}/mature_biomass_area_2020")


# Most efficient - uses reduceRegion with bestEffort
total_biomass = mature_biomass.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mature_biomass.geometry(),
    scale=mature_biomass.projection().nominalScale(),
    maxPixels=1e9,
    bestEffort=True
).getInfo()

print(f"Total biomass: {total_biomass}")

Total biomass: {'mature_biomass': 19499281737}


In [ ]:
# Load the categorical image and select the 'biome' band
biomes = ee.Image(f"{data_folder}/categorical").select("biome")
biomes_mask = biomes.eq(1).rename("biome_mask")
lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
            .select([f"classification_{year}" for year in config.range_1985_2020])
            .byte()
            .rename([str(year) for year in config.range_1985_2020]))
mature_mask = lulc.eq(3).reduce(ee.Reducer.allNonZero()).unmask(0).updateMask(biomes_mask)

# Get area coverage in million hectares
pixel_area = ee.Image.pixelArea()
valid_pixels = mature_mask
covered_area = pixel_area.updateMask(valid_pixels).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mature_biomass.geometry(),
    scale=mature_biomass.projection().nominalScale(),
    maxPixels=1e9,
    bestEffort=True
).getInfo()

# Convert from m² to million hectares
# 1 hectare = 10,000 m²
# 1 million hectares = 10,000,000,000 m²
area_million_hectares = covered_area['area'] / 1e10

print(f"Covered area: {area_million_hectares:.2f} million hectares")

Covered area: 422.36 million hectares


In [ ]:
# Load the categorical image and select the 'biome' band
biomes = ee.Image(f"{data_folder}/categorical").select("biome")
biomes_mask = biomes.eq(1).rename("biome_mask")
lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
            .select([f"classification_{year}" for year in config.range_1985_2020])
            .byte()
            .rename([str(year) for year in config.range_1985_2020]))
mature_mask = lulc.eq(15).reduce(ee.Reducer.allNonZero()).unmask(0).updateMask(biomes_mask)

# Get area coverage in million hectares
pixel_area = ee.Image.pixelArea()
valid_pixels = mature_mask
covered_area = pixel_area.updateMask(valid_pixels).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mature_biomass.geometry(),
    scale=mature_biomass.projection().nominalScale(),
    maxPixels=1e9,
    bestEffort=True
).getInfo()

# Convert from m² to million hectares
# 1 hectare = 10,000 m²
# 1 million hectares = 10,000,000,000 m²
area_million_hectares = covered_area['area'] / 1e10

print(f"Covered area: {area_million_hectares:.2f} million hectares")

KeyboardInterrupt: 